In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import QUBO_SVM_functions as qSVM
import kernel_SVM_functions as kSVM

from parameter_tuning import tune_qsvm_parameters
from QP_cross_validation import qp_cross_validate
from plot_boundary import plot

In [2]:
with open('synth_data/synth_0.3.csv', 'r') as f:
    data = np.loadtxt(f, delimiter=',')

X = data[:, :-1]
t = data[:, -1]

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size = 0.8, shuffle = True, stratify = t)
t_train = t_train.reshape(-1, 1)
t_test = t_test.reshape(-1, 1)

In [3]:
#model params
B = 2
K = 3
R = 3
kernel_func = kSVM.rbf_kernel
gamma = 4
#Annealing params
cooling_param = 6
m_rate = 1 / K
iterations = 1000
T0 = 20
#cross_validation params
k_folds = 10


In [4]:
qubo_model = qSVM.QUBOSoftMarginClassifier(B, K, R, kernel_func, gamma)
qubo_model = qubo_model.set_annealing_parameters(cooling_param, m_rate, iterations, T0)

In [5]:
train_results, test_results = qp_cross_validate(X_train, t_train, k_folds, qubo_model, num_models = 10)
print(f"The score for this model is: \n {np.mean(test_results)}")

The score for this model is: 
 0.9418000000000001


In [6]:
B_values = [2, 3, 5, 10]
K_values = [2, 3]
R_values = [0, 1, 3, 5]
gamma_values = [-1] + [2 ** i for i in range(-3, 3)]

In [7]:
results, times = tune_qsvm_parameters(X_train, t_train, B_values, K_values, R_values, gamma_values, kernel_func)

9.82611083984375
8.45782995223999
8.97814393043518
10.518081903457642
11.137022972106934
9.370709896087646
9.616585969924927
8.819469690322876
8.683484077453613
9.146410942077637
9.606812238693237
9.414103984832764
9.602023839950562
8.591707944869995
9.183170080184937
9.241620063781738
8.559144973754883
9.313392877578735
8.921651124954224
9.478616714477539
8.924914121627808
9.179965734481812
8.608019351959229
9.002506971359253
9.049258947372437
8.627530097961426
8.482847929000854
9.754483938217163
11.329826831817627
10.031687259674072
9.46177887916565
10.857348918914795
9.402708053588867
9.976277828216553
10.419264078140259
9.753257989883423
9.703105926513672
9.420025825500488
9.730343103408813
10.026566743850708
9.439696073532104
10.460263729095459
11.000728130340576
9.312988996505737
9.900170087814331
9.291953086853027
10.017151832580566
10.094173908233643
9.481569051742554
9.886678218841553
9.770186185836792
9.237567901611328
9.957082986831665
9.949475049972534
9.455871105194092
9.9

In [10]:
np.save('QUBO_parameter_tuning_results', results)
np.save('QUBO_parameter_tuning_times', times)